In [28]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels


import numpy as np
import plotly.graph_objects as go

# GPR Example

## Training Data

In [25]:
def noisy_sine(x: np.ndarray):
    """
    Produce a noisy sine function.
    
    Parameters
    ----------
    x : np.ndarray
            x data at which to compute the 
            function.
    """
    return np.sin(x) + 0.1 * np.random.normal(size=x.shape)

In [26]:
x = np.linspace(0, 10, 100)
y = noisy_sine(x)

In [27]:
fig = px.scatter(x=x, y=y)
fig.show()

## Prior

In [74]:
def plot_samples(kernel: str, n_samples: int, model: GaussianProcessRegressor = None):
    """
    Plot samples from the Gaussian process model.
    
    Parameters
    ----------
    kernel : str
            Kernel used in the multivariate distribution.
            Options include:
                - squared-exponential
                - periodic
                - identity
                - matern
                - dot-product
    n_samples : int
            Number of samples to plot.
    model : GaussianProcessRegressor
            Model to be used instead of a new one being constructed.
    """
    kernel_options = {
        "squared-exponential": 1.0 * kernels.RBF(
            length_scale=1.0, length_scale_bounds=(1e-1, 10.0)
        ),
        "periodic": 1.0 * kernels.ExpSineSquared(
            length_scale=1.0,
            periodicity=3.0,
            length_scale_bounds=(0.1, 10.0),
            periodicity_bounds=(1.0, 10.0)
        ),
        "identity": kernels.ConstantKernel(constant_value=1.0),
        "matern": 1.0 * kernels.Matern(
            length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5
        ),
        "dot-product": kernels.DotProduct(
            sigma_0=1.0, sigma_0_bounds=(0.1, 10.2)
        )**2
    }
    
    if model is None:
        model = GaussianProcessRegressor(
            kernel=kernel_options[kernel], random_state=0
        )
    
    x = np.linspace(0, 10, 100)
    X = x.reshape(-1, 1)
    
    y_mean, y_std = model.predict(X, return_std=True)
    y_samples = model.sample_y(X, n_samples)
    
    fig = go.Figure()
    # Plot the priors
    for prior_draw in y_samples.T:
        fig.add_trace(
            go.Scatter(x=x, y=prior_draw, mode="lines")
        )
    # Plot the mean function
    fig.add_trace(
        go.Scatter(
            x=x, 
            y=y_mean, 
            mode="lines", 
            line=dict(color="black", width=4)
        )
    )
    # Plot the error bands over the mean function
    y_upper = y_mean + y_std
    y_lower = y_mean - y_std
    fig.add_trace(
        go.Scatter(
            x=x + x[::-1],
            y = y_upper + y_lower[::-1],
            fill="toself",
            fillcolor="rgba(0, 100, 80, 0.2)",
            line=dict(color="rgba(255, 255, 255, 0)"),
            hoverinfo="skip",
        )
    )
    fig.update_layout(showlegend=False)
    fig.show()


In [75]:
plot_samples("squared-exponential", n_samples=50)

### Training A Model

In [76]:
kernel = kernels.RBF(
            length_scale=1.0, length_scale_bounds=(1e-1, 10.0)
        )
model = GaussianProcessRegressor(kernel=kernel, random_state=0)

In [ ]:
model.fit(X)